In [248]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

E:\miniconda\envs\course_py35\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# groupby mechanics

## 分类统计

In [3]:
df1 = pd.DataFrame({'key1':['a','a','b','b','a'],
                    'key2':['one','two','one','two','one'],
                    'data1':np.random.randn(5),
                    'data2':np.random.randn(5)})
df1

,data1,data2,key1,key2
0,-0.048543,-0.022294,a,one
1,-2.138261,-0.755281,a,two
2,-1.089498,0.055245,b,one
3,-0.942707,-0.269355,b,two
4,-1.319833,-1.617892,a,one


In [14]:
mean1 = df1['data1'].groupby([df1['key1'],df1['key2']]).mean()
mean1

key1  key2
a     one    -0.684188
      two    -2.138261
b     one    -1.089498
      two    -0.942707
Name: data1, dtype: float64

In [17]:
df1[['key1','data1','key2']].groupby(by = ['key1','key2'],as_index = True,group_keys = False).mean().unstack()  

data1          
key2       one       two
key1                    
a    -0.684188 -2.138261
b    -1.089498 -0.942707

In [15]:
mean1.unstack()

key2,one,two
key1,,
a,-0.684188,-2.138261
b,-1.089498,-0.942707


In [19]:
states = ['ohio','california','california','ohio','ohio']
df1['data1'].groupby(by = [states,np.array([2005,2005,2006,2005,2006])]).mean()

california  2005   -2.138261
            2006   -1.089498
ohio        2005   -0.495625
            2006   -1.319833
Name: data1, dtype: float64

In [29]:
df1.groupby(by = ['key1','key2']).count() # 更简单的方法

data1  data2
key1 key2              
a    one       2      2
     two       1      1
b    one       1      1
     two       1      1

In [32]:
df1.groupby(by = ['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## 迭代分出来的类

In [34]:
for name,group in df1.groupby(by = ['key1','key2']):
    print(name)
    print(group)

('a', 'one')
      data1     data2 key1 key2
0 -0.048543 -0.022294    a  one
4 -1.319833 -1.617892    a  one
('a', 'two')
      data1     data2 key1 key2
1 -2.138261 -0.755281    a  two
('b', 'one')
      data1     data2 key1 key2
2 -1.089498  0.055245    b  one
('b', 'two')
      data1     data2 key1 key2
3 -0.942707 -0.269355    b  two


In [39]:
pieces = dict(list(df1.groupby('key1')))
pieces['b']

,data1,data2,key1,key2
2,-1.089498,0.055245,b,one
3,-0.942707,-0.269355,b,two


In [41]:
df1.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [46]:
list(df1.groupby(by = df1.dtypes,axis = 1))[0][1]

,data1,data2
0,-0.048543,-0.022294
1,-2.138261,-0.755281
2,-1.089498,0.055245
3,-0.942707,-0.269355
4,-1.319833,-1.617892


In [50]:
df1.groupby('key1')['data1'] is df1.groupby('key1')[['data1']]#和[['data1']] 的结果一样

False

In [51]:
df2 = pd.DataFrame(np.random.randn(6,4),
                   columns = pd.Index(['a','b','c','d']),
                   index = ['one','two','three','four','five','six'])

In [52]:
df2

,a,b,c,d
one,0.011484,0.383645,-0.355455,0.169154
two,-0.792145,-0.532953,-0.982333,1.299576
three,-0.735642,-0.481439,-1.128822,-1.803897
four,-0.943761,-1.252705,-0.357421,0.487231
five,-0.344771,0.373569,0.229179,-0.203846
six,0.611310,-0.214866,0.100822,0.169901


In [57]:
mapping = {'a':'red','b':'blue','c':'red','d':'blue','e':'yellow'}
mapping2 = {'one':'small','two':'small','three':'small','four':'large'}
df2.groupby(mapping,axis = 1).mean()

,blue,red
one,0.276399,-0.171985
two,0.383312,-0.887239
three,-1.142668,-0.932232
four,-0.382737,-0.650591
five,0.084862,-0.057796
six,-0.022483,0.356066


In [60]:
df2.groupby(mapping2).mean().groupby(mapping,axis =1).mean()

,blue,red
large,-0.382737,-0.650591
small,-0.160986,-0.663819


In [61]:
s1 = pd.Series(mapping)
s1

a       red
b      blue
c       red
d      blue
e    yellow
dtype: object

In [63]:
df2.groupby(s1,axis=1).mean()

,blue,red
one,0.276399,-0.171985
two,0.383312,-0.887239
three,-1.142668,-0.932232
four,-0.382737,-0.650591
five,0.084862,-0.057796
six,-0.022483,0.356066


In [66]:
df2.groupby(len).count()

,a,b,c,d
3,3,3,3,3
4,2,2,2,2
5,1,1,1,1


In [72]:
df2.groupby(by = [mapping2,len]).count()

a  b  c  d
large 4  1  1  1  1
small 3  2  2  2  2
      5  1  1  1  1

In [94]:
df3 = pd.DataFrame(np.random.normal(0,1,(5,5)),
                   index = pd.MultiIndex.from_arrays(
                       [['US','US','US','JP','JP'],[1,3,5,1,3]],
                   names = ['cty','tensor']))

In [100]:
df3.groupby(level = 'tensor',axis = 0).size()

tensor
1    2
3    2
5    1
dtype: int64

# 数据聚合

In [104]:
df1.groupby(by = 'key1').quantile(0.8) # 求分位点

0.8,data1,data2
key1,,
a,-0.557059,-0.315489
b,-0.972065,-0.009675


In [105]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
df1.groupby(by = 'key1').agg(peak_to_peak)

,data1,data2
key1,,
a,2.089718,1.595599
b,0.146791,0.324600


In [107]:
df4 = pd.read_csv('examples\\tips.csv')
df4['tip_pct'] = df4['tip'].div(df4.total_bill)

In [108]:
df4.sample(4)

,total_bill,tip,smoker,day,time,size,tip_pct
117,10.65,1.50,No,Thur,Lunch,2,0.140845
223,15.98,3.00,No,Fri,Lunch,3,0.187735
113,23.95,2.55,No,Sun,Dinner,2,0.106472
116,29.93,5.07,No,Sun,Dinner,4,0.169395


In [110]:
grouped = df4.groupby(by = ['day','smoker'])

In [113]:
grouped['tip_pct'].aggregate(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [116]:
grouped.tip_pct.aggregate([('期望值','mean'),('标准差',np.std)])

期望值       标准差
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [117]:
grouped['total_bill','tip_pct'].aggregate(['count','mean',np.sum])

total_bill                     tip_pct                    
                 count       mean      sum   count      mean       sum
day  smoker                                                           
Fri  No              4  18.420000    73.68       4  0.151650  0.606602
     Yes            15  16.813333   252.20      15  0.174783  2.621746
Sat  No             45  19.661778   884.78      45  0.158048  7.112145
     Yes            42  21.276667   893.62      42  0.147906  6.212055
Sun  No             57  20.506667  1168.88      57  0.160113  9.126438
     Yes            19  24.120000   458.28      19  0.187250  3.557756
Thur No             45  17.113111   770.09      45  0.160298  7.213414
     Yes            17  19.190588   326.24      17  0.163863  2.785676

In [121]:
grouped['total_bill','tip_pct'].aggregate({'total_bill':['sum','count'],
                                           'tip_pct':['mean']}).reset_index()

day smoker   tip_pct total_bill      
                    mean        sum count
0   Fri     No  0.151650      73.68     4
1   Fri    Yes  0.174783     252.20    15
2   Sat     No  0.158048     884.78    45
3   Sat    Yes  0.147906     893.62    42
4   Sun     No  0.160113    1168.88    57
5   Sun    Yes  0.187250     458.28    19
6  Thur     No  0.160298     770.09    45
7  Thur    Yes  0.163863     326.24    17

# apply

In [125]:
def top(df,n = 5,column = 'tip_pct'):
    return df.sort_values(by = column,ascending = False)[:n]

In [123]:
top(df4)

,total_bill,tip,smoker,day,time,size,tip_pct
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [126]:
df4.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [127]:
df4.groupby(['smoker','day']).apply(top,n=1,column = 'total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [134]:
df5 = df4.groupby(['smoker'])['tip_pct'].describe()
df5

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [135]:
df5.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [139]:
f = lambda x :x.describe()
df4.groupby(['smoker'])['tip_pct'].apply(f).unstack()

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [144]:
df4.groupby(['smoker'],group_keys = False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [145]:
df6 = pd.DataFrame(np.random.normal(0,1,(100,2)),
                  columns = ['data1','data2'])

In [147]:
quartiles = pd.cut(df6.data1,4)

In [152]:
def get_stats(df):
    return {'min':df.mean(),'max':df.max(),
            'count':df.count(),'mean':df.mean()}

In [167]:
df6.groupby(quartiles).data2.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.74, -1.382]",6.0,1.346942,0.193849,0.193849
"(-1.382, -0.0294]",39.0,1.876303,0.105957,0.105957
"(-0.0294, 1.323]",43.0,2.547063,0.115097,0.115097
"(1.323, 2.676]",12.0,1.577175,-0.009655,-0.009655


In [169]:
qquatiles = pd.qcut(df6.data1,4)

In [170]:
df6.groupby(qquatiles).data2.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.735, -0.608]",25.0,1.876303,-0.009570,-0.009570
"(-0.608, 0.0855]",25.0,2.361456,0.342511,0.342511
"(0.0855, 0.794]",25.0,2.547063,0.166296,0.166296
"(0.794, 2.676]",25.0,1.577175,-0.094087,-0.094087


In [176]:
s2 = pd.Series(np.arange(5))
s2[::2] = np.nan

In [179]:
s2.fillna(s2.mean())

0    2.0
1    1.0
2    2.0
3    3.0
4    2.0
dtype: float64

In [182]:
states = list('abcdefgh')
keys = ['west'] * 4 + ['east'] * 4
s3 = pd.Series(np.random.randn(8),index = states)
s3[::2] = np.nan


In [183]:
f = lambda g :g.fillna(g.mean())
s3.groupby(by = keys).apply(f)

a    1.007558
b    0.794237
c    1.007558
d    1.220879
e    0.296912
f   -0.342952
g    0.296912
h    0.936775
dtype: float64

In [186]:
fill_value = {'west':0.5,'east':-1}
f2 = lambda g :g.fillna(fill_value[g.name])
s3.groupby(by = keys).apply(f2)

a    0.500000
b    0.794237
c    0.500000
d    1.220879
e   -1.000000
f   -0.342952
g   -1.000000
h    0.936775
dtype: float64

In [191]:
suits = ['H','S','C','D'] #Hearts, Spades,Clubs,Diomands
cards = []
card_val = (list(range(1,11)) + [10]*3) *4
base_names = ['A'] + list(range(2,11)) + ['J','K','Q']
cards = [str(num) + suit  for suit in suits for num in base_names]
deck = pd.Series(card_val,index = cards)
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [189]:
def draw(deck,n = 5):
    return deck.sample(n)

In [192]:
draw(deck)

3H      3
3C      3
10C    10
9S      9
7D      7
dtype: int64

In [195]:
f = lambda g:g[-1]
deck.groupby(by = f).apply(draw,n=2)

C  9C      9
   JC     10
D  5D      5
   QD     10
H  4H      4
   JH     10
S  10S    10
   QS     10
dtype: int64

In [198]:
df7 = pd.DataFrame({'category':list('aaabbbab'),
                    'weights':np.random.rand(8),
                    'data':np.random.randn(8)})

In [207]:
f4 = lambda g:np.average(g['data'],weights = g.weights)

In [209]:
df7.groupby(by = 'category').apply(f4) # groupby 生成的是在同一列中的不同pieces

category
a    0.218255
b   -0.195883
dtype: float64

In [212]:
df8 = pd.read_csv('examples\\stock_px_2.csv',parse_dates = True,
                  index_col = 0)
df8.sample(8)

,AAPL,MSFT,XOM,SPX
2003-09-25,10.22,22.26,31.24,1003.27
2003-11-03,11.57,21.15,30.84,1059.02
2011-05-27,337.41,24.60,82.09,1331.10
2005-08-15,47.68,24.27,53.19,1233.87
2004-10-13,19.88,22.29,41.82,1113.65
2003-11-26,10.36,20.18,30.72,1058.45
2007-10-24,185.93,28.73,84.16,1515.88
2009-04-14,118.31,18.30,63.63,841.50


In [214]:
df8.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [241]:
df9.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2213 entries, 2003-01-03 to 2011-10-14
Data columns (total 4 columns):
AAPL    2213 non-null float64
MSFT    2213 non-null float64
XOM     2213 non-null float64
SPX     2213 non-null float64
dtypes: float64(4)
memory usage: 86.4 KB


In [215]:
spx_corr = lambda x :x.corrwith(x['SPX'])

In [244]:
df9 = df8.pct_change().dropna()
df9.sample(8) 

,AAPL,MSFT,XOM,SPX
2010-09-07,-0.003710,-0.013508,-0.012525,-0.011471
2003-05-05,0.114958,-0.009232,-0.016393,-0.003795
2008-06-20,-0.031122,-0.024173,-0.010207,-0.018543
2008-09-25,0.025017,0.034583,0.033908,0.019656
2007-05-21,0.017815,0.007436,0.003960,0.001543
2010-01-14,-0.005792,0.020219,0.000150,0.002427
2004-06-09,-0.005270,-0.004742,-0.010096,-0.009499
2006-12-13,0.033782,0.004108,0.014539,0.001169


In [219]:
get_year = lambda x :x.year

In [240]:
df8.groupby(by = get_year).apply(spx_corr).merge(df9.groupby(by = get_year).apply(
    spx_corr),left_index = True,right_index = True)[[
    'AAPL_x','AAPL_y','XOM_x','XOM_y']]    

,AAPL_x,AAPL_y,XOM_x,XOM_y
2003,0.882692,0.541124,0.860104,0.661265
2004,0.638080,0.374283,0.432642,0.557742
2005,0.743263,0.467540,0.404042,0.631010
2006,0.838252,0.428267,0.849528,0.518514
2007,0.483093,0.508118,0.729426,0.786264
2008,0.829417,0.681434,0.774885,0.828303
2009,0.952895,0.707103,0.278447,0.797921
2010,0.617886,0.710105,0.894656,0.839057
2011,-0.552395,0.691931,0.912603,0.859975


In [243]:
df9.groupby(by = get_year).apply(lambda g:g['AAPL'].corr(g['MSFT'])) 

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

In [252]:
def regression(data,yvar,xvar):
    y = data[yvar]
    x = data[xvar]
    x = sm.add_constant(x)
#     x['intercept'] = 1   解释了上面这一句
    result = sm.OLS(y,x).fit()
    return result.params

In [253]:
df9.groupby(get_year).apply(regression,'AAPL',['SPX'])

,const,SPX
2003,0.000710,1.195406
2004,0.004201,1.363463
2005,0.003246,1.766415
2006,0.000080,1.645496
2007,0.003438,1.198761
2008,-0.001110,0.968016
2009,0.002954,0.879103
2010,0.001261,1.052608
2011,0.001514,0.806605


# pivot and cross tabulation

In [259]:
df4.sample(8)

,total_bill,tip,smoker,day,time,size,tip_pct
91,22.49,3.50,No,Fri,Dinner,2,0.155625
62,11.02,1.98,Yes,Sat,Dinner,2,0.179673
52,34.81,5.20,No,Sun,Dinner,4,0.149382
230,24.01,2.00,Yes,Sat,Dinner,4,0.083299
24,19.82,3.18,No,Sat,Dinner,2,0.160444
197,43.11,5.00,Yes,Thur,Lunch,4,0.115982
59,48.27,6.73,No,Sat,Dinner,4,0.139424
234,15.53,3.00,Yes,Sat,Dinner,2,0.193175


In [257]:
df4.pivot_table(index = ['day','smoker']) # 和下面的结果一样

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [260]:
df4.groupby(['day','smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [261]:
df4.pivot_table(['tip_pct','size'],index = ['time','day'],columns = 'smoker') 
#这时用groupby就搞不定了

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [263]:
df4.pivot_table(['tip_pct','size'],index = ['time','day'],columns = 'smoker',
               margins = True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [268]:
df4.pivot_table(['tip_pct'],index = ['time','smoker'],columns = 'day',
               margins = True,fill_value = 0,aggfunc = len) # 默认是mean

tip_pct                         
day               Fri   Sat   Sun  Thur    All
time   smoker                                 
Dinner No         3.0  45.0  57.0   1.0  106.0
       Yes        9.0  42.0  19.0   0.0   70.0
Lunch  No         1.0   0.0   0.0  44.0   45.0
       Yes        6.0   0.0   0.0  17.0   23.0
All              19.0  87.0  76.0  62.0  244.0

In [269]:
pd.crosstab([df4.time,df4.smoker],df4.day,margins = True) # 也可以用pivot_table

day            Fri  Sat  Sun  Thur  All
time   smoker                          
Dinner No        3   45   57     1  106
       Yes       9   42   19     0   70
Lunch  No        1    0    0    44   45
       Yes       6    0    0    17   23
All             19   87   76    62  244

In [271]:
df4.pivot_table(['tip_pct'],index = ['time','smoker'],columns ='day',
                aggfunc = 'count',margins = True)

tip_pct                         
day               Fri   Sat   Sun  Thur    All
time   smoker                                 
Dinner No         3.0  45.0  57.0   1.0  106.0
       Yes        9.0  42.0  19.0   NaN   70.0
Lunch  No         1.0   NaN   NaN  44.0   45.0
       Yes        6.0   NaN   NaN  17.0   23.0
All              19.0  87.0  76.0  62.0  244.0